In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = False)
#browser = Browser('chrome')

In [3]:
#important variables saved in this assignment
'''
news_title
news_p
featured_image_url
'''
print('')

In [4]:
mars_url = 'https://mars.nasa.gov/news/'
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
weather_url = 'https://twitter.com/marswxreport?lang=en'
twt_url = 'https://twitter.com/marswxreport?lang=en'
facts_url = 'https://space-facts.com/mars/'

In [5]:
browser.visit(mars_url)

In [7]:
html = browser.html
soup = bs(html, 'html.parser')
#scraping NASA Mars News for newest article
news_title = soup.select('div.content_title > a')[0].text
news_p = soup.select('div.article_teaser_body')[0].text

In [ ]:
#when i restart and run all cells, i get "list index out of range" when assigning news_p.
#if i rerun the cell above I don't get that problem. Will look for fix but moving on for now.

In [8]:
#visiting JPL webpage
#jpl_link is used for str concatenation after finding img link
browser.visit(jpl_url)
img_html = browser.html
soup = bs(img_html, 'html.parser')
jpl_link = 'https://www.jpl.nasa.gov'

In [9]:
#locating tag with featured img
foot_tag = soup.find('footer')
#foot_tag

In [10]:
img_link = foot_tag.contents[1]['data-fancybox-href']

In [11]:
featured_image_url = jpl_link + img_link
#featured_image_url
browser.visit(featured_image_url)

In [12]:
browser.visit(twt_url)

In [13]:
html_weather = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html_weather, 'html.parser')

# Find all elements that contain tweets
tweets = soup.select("div.css-1dbjc4n > article")[0].text

In [14]:
tweets

'Mars Weather@MarsWxReport·May 29InSight sol 535 (2020-05-29) low -91.3ºC (-132.4ºF) high -2.7ºC (27.2ºF)\nwinds from the SW at 5.2 m/s (11.5 mph) gusting to 16.7 m/s (37.3 mph)\npressure at 7.20 hPa830'

In [15]:
tweet = tweets[29:]

In [16]:
tweet

' 29InSight sol 535 (2020-05-29) low -91.3ºC (-132.4ºF) high -2.7ºC (27.2ºF)\nwinds from the SW at 5.2 m/s (11.5 mph) gusting to 16.7 m/s (37.3 mph)\npressure at 7.20 hPa830'

In [17]:
browser.visit(facts_url)

In [18]:
html_facts = browser.html
soup = bs(html_facts, 'html.parser')

In [19]:
facts= soup.find('table')

In [21]:
facts.text

'Equatorial Diameter:6,792 kmPolar Diameter:6,752 kmMass:6.39 × 10^23 kg (0.11 Earths)Moons:2 (Phobos & Deimos)Orbit Distance:227,943,824 km (1.38 AU)Orbit Period:687 days (1.9 years)Surface Temperature: -87 to -5 °CFirst Record:2nd millennium BCRecorded By:Egyptian astronomers'